<h2> Augmented cluster generation on the basis of cluster.json</h2>

Requirements:
1. cluster.json : https://drive.google.com/file/d/1FXM9SxGJ7o_DM_l7__5T7GxFywngxhi0/view?usp=share_link



In [ ]:
! pip install transformers datasets huggingface_hub

# logging in
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# All installations here


from google.colab import drive
drive.mount('/content/drive')

# git installation
!apt install git-lfs
!git config --global user.email "nandysoham16@gmail.com"
!git config --global user.name "nandysoham"


# all imports
import tensorflow as tf
import transformers
from datasets import load_dataset, load_metric
import pandas as pd
import numpy as np
import random
from transformers import AutoTokenizer
import transformers
from transformers import TFAutoModelForQuestionAnswering
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard
import collections
from transformers import create_optimizer

import json
with open('/content/drive/MyDrive/cluster.json') as json_file:
    clusters_dict = json.load(json_file)


clustered = clusters_dict["clusters"]
centroids = clusters_dict["centroids"]

Mounted at /content/drive
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.


In [ ]:
# parameters for the data and the model
squad_v2 = False
batch_size = 16
model_checkpoint = "Rocketknight1/distilbert-base-uncased-finetuned-squad"

datasets = load_dataset("squad_v2" if squad_v2 else "squad",  split='train')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


In [ ]:
clustered["0"]

['Canadian_Armed_Forces',
 'United_Nations_Population_Fund',
 'National_Archives_and_Records_Administration',
 'Arena_Football_League',
 'Ministry_of_Defence_(United_Kingdom)',
 'Identity_(social_science)',
 'Federal_Aviation_Administration',
 'Military_history_of_the_United_States',
 'Federal_Bureau_of_Investigation',
 'Anti-aircraft_warfare',
 'United_States_Army',
 'Freemasonry',
 'Nonprofit_organization',
 'Affirmative_action_in_the_United_States',
 'European_Central_Bank',
 'Group_(mathematics)',
 'Federalism',
 'Central_Intelligence_Agency',
 'Association_football',
 'United_States_Air_Force']

In [ ]:
datasets

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [ ]:
df_squad_v1 = pd.DataFrame( datasets)

In [ ]:
df_squad_v1

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...
...,...,...,...,...,...
87594,5735d259012e2f140011a09d,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,"{'text': ['Oregon'], 'answer_start': [229]}"
87595,5735d259012e2f140011a09e,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,"{'text': ['Rangoon'], 'answer_start': [414]}"
87596,5735d259012e2f140011a09f,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,"{'text': ['Minsk'], 'answer_start': [476]}"
87597,5735d259012e2f140011a0a0,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,"{'text': ['1975'], 'answer_start': [199]}"


In [ ]:
# -----------------All other models that are in use ----------------------#
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer


clustering_model = SentenceTransformer('all-MiniLM-L6-v2')

# 40S
!pip install nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

#  ref: https://github.com/makcedward/nlpaug/blob/master/example/tfidf-train_model.ipynb

from nlpaug.util import Action

import os
os.environ["MODEL_DIR"] = '../model'
model_dir = '../model'


import sklearn.datasets
import re

import nlpaug.augmenter.word as naw
import nlpaug.model.word_stats as nmw

def _tokenizer(text, token_pattern=r"(?u)\b\w\w+\b"):
    token_pattern = re.compile(token_pattern)
    return token_pattern.findall(text)

# Load sample data
train_data = sklearn.datasets.fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
train_x = train_data.data

# Tokenize input
train_x_tokens = [_tokenizer(x) for x in train_x]

# Train TF-IDF model
tfidf_model = nmw.TfIdf()
tfidf_model.train(train_x_tokens)
tfidf_model.save('.')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=47043de4e7c261a741e42d3f0d4b3a46f278c98af0f6f187700d5773b9bdca46
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 KB 21.5 MB/s eta 0:00:00


In [ ]:

augmenter = ["delete", "wordnet","word2vec", "TFID", "antonym", "BERT"]


"""
TfID augmenter
"""
def augment_text_TFID(text):
  # aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")
 # Load TF-IDF augmenter
  aug = naw.TfIdfAug(model_path='.', tokenizer=_tokenizer, action="substitute")  # insert, substitute and other options exist
  augmented_text = aug.augment(text)
  # print("Original:")
  # print(text)
  # print("Augmented Text:")
  # print(augmented_text)
  return augmented_text

"""
synonym generator
"""
def augment_text_wordnet(text):
  aug = naw.SynonymAug(aug_src='wordnet')
  augmented_text = aug.augment(text)
  return augmented_text

def augment_text_word2vec(text):
  aug = naw.WordEmbsAug(
    model_type='word2vec', model_path='/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300',
    action="substitute")
  augmented_text = aug.augment(text)
  return augmented_text

def augment_bert(text):
  aug = nas.ContextualWordEmbsForSentenceAug(model_path='xlnet-base-cased')
  augmented_texts = aug.augment(text, n=3)
  return augmented_texts

def augment_text_antonym(text):
  aug = naw.AntonymAug()
  augmented_text = aug.augment(text)
  return augmented_text

def augment_text_delete(text):
  aug = naw.RandomWordAug()
  augmented_text = aug.augment(text)
  return augmented_text


def augment_text(text):

  rand_no = random.randint(2,len(augmenter))
  augmented_list = random.choices(augmenter, k = rand_no)
  # print(augmented_list)
  for layer in augmenter:
    if(layer == "wordnet"):
      text = augment_text_wordnet(text)

    if(layer == "TFID"):
      text = augment_text_TFID(text)

    if(layer == "word2vec"):
      test = augment_text_word2vec(text)

    if(layer == "antonym"):
      text = augment_text_antonym(text)

    if(layer == "delete"):
      text = augment_text_delete(text)

    if(layer == "BERT"):
      text = augment_bert(text)
    return text
  # print("error in program")

In [ ]:
# parameters for the data and the model
squad_v2 = False
batch_size = 16
model_checkpoint = "Rocketknight1/distilbert-base-uncased-finetuned-squad"


from traitlets.config import T
import numpy as np

# 140 S
def Generate_Synthetic_Data(theme_df,theme, count):

  synthetic_list = []
  synthetic_list_augmented =[]
  extra_item = 0
  force_stop = False  # if in case no new dats can be generated

  while(len(synthetic_list) < count and not force_stop):
    # print("Here")
    # print(len(synthetic_list))
    stop = True
    for ind in theme_df.index:
      # print(ind)
      if(len(synthetic_list) > count):
        break

      context = theme_df["context"][ind]
      isposs = True
      answer_start = -1
      answer_end = -1
      text = ""
      question = theme_df["question"][ind]
      if(len(theme_df["answers"][ind]["answer_start"]) > 0):
        text = theme_df["answers"][ind]["text"][0]
        text_size = len(text)
        answer_start = theme_df["answers"][ind]["answer_start"][0]
        answer_end = answer_start + text_size - 1
      else:
        isposs = False

      # print(context)
      # print(text)
      # print(answer_start)
      # print(answer_end)

      # if(isposs):
      # problem here 12.35 --> will also get split
      sentence_list = context.split(".")

      # breaking the whole sentence
      # print(sentence_list)

      # each entry --> [start_index_of_the_sentence , end_index_of_the_sentence(includes .)]
      sentence_index_list = []

      # mask[0] = 0 --> answer not present, else present
      sentence_mask = []
      prev = 0;


      for sentence in sentence_list:
        sentence_index_list.append([prev, prev+len(sentence)])
        prev = prev+len(sentence)+1

      # ============= open up to debug ==============
      # print(sentence_index_list)

      # for lst in sentence_index_list:
      #   print(context[lst[0]:lst[1] + 1], sep=",")
      # print()

      for pr in sentence_index_list:
        if(pr[0] > answer_end or pr[1] < answer_start):
          sentence_mask.append(0)
        else:
          sentence_mask.append(1)

      # print(sentence_mask)
      n = len(sentence_mask)

      # these are final after combining every sentence where mask = 1
      sentence_mask_final = []
      sentence_index_list_final = []
      i =0
      offset = 0

      # combining the ones together
      while(i < n):
        if(sentence_mask[i] == 0):
          sentence_mask_final.append(0)
          sentence_index_list_final.append(sentence_index_list[i])
        else:
          start_idx = sentence_index_list[i][0]
          offset = answer_start - start_idx
          end_idx = sentence_index_list[i][1]
          while(i < n and sentence_mask[i] == 1):
            end_idx = sentence_index_list[i][1]
            i+=1
          if(i < n):
            i-=1
          sentence_mask_final.append(1)
          sentence_index_list_final.append([start_idx, end_idx])
        i+=1
      # print(sentence_mask_final)
      # print(sentence_index_list_final)

      n = len(sentence_mask_final)
      reordered = np.arange(n)
      np.random.shuffle(reordered)
      # print(reordered)
      corpus = ""

      # using nlpaug to generate augmented data
      ans_index = -1
      before_ans = ""
      after_ans = ""
      actual_ans_sentence = ""

      if(isposs):
        before = 0
        passed_answer = False
        for lst in reordered:
          if(not passed_answer):
            if(sentence_mask_final[lst] == 0):
              before += sentence_index_list_final[lst][1] - sentence_index_list_final[lst][0] + 1
              before_ans += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]
            else:
              ans_index = before + offset
              actual_ans_sentence += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]
              passed_answer = True

          else:
            after_ans += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]
          corpus += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]
        pass
      else:
        for lst in reordered:
          corpus += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]
          before_ans += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]


      # print("before ans --> " ,before_ans)
      # print("after_ans -->" , after_ans)
      # print("actual ans sentence --> ",actual_ans_sentence)

      before_ans = augment_text(before_ans)[0] if (len(augment_text(before_ans)) > 0 )  else ""
      after_ans = augment_text(after_ans)[0] if (len(augment_text(after_ans)) > 0 )  else ""

      augmented_ans_index = -1
      if(len(text)):
        augmented_ans_index = len(before_ans) + offset

      augmented_corpus = before_ans + actual_ans_sentence + after_ans
      entry = {}
      entry["id"] = str(np.random.rand()*1e5)
      entry["title"] = theme
      entry["context"] = augmented_corpus
      entry["question"] = question
      entry["answers"] = {"text" : [text if len(text) else ""], "answer_start": [augmented_ans_index] if augmented_ans_index > -1 else []}
      # print("org ans = ", text)
      # if(augmented_ans_index != -1):
      #   print("ans = ", augmented_corpus[augmented_ans_index : augmented_ans_index + len(text)])
      # print(entry)

      synthetic_list_augmented.append(entry)
      ######## ------------- open this up to create just normal - non augmented data --------------#########
      entry = {}
      entry["id"] = str(np.random.rand()*1e5)
      entry["title"] = theme
      entry["context"] = corpus
      entry["question"] = question
      entry["answers"] = {"text" : [text if len(text) else ""], "answer_start": [ans_index] if ans_index > -1 else []}

      # print("-----------------------------")
      # print(context)
      # print(corpus)
      # print(text)
      # print(ans_index)
      # print(corpus[ans_index : ans_index + len(text)])
      # print("-----------------------------")

      prev_len = len(synthetic_list)
      synthetic_list.append(entry)
      up_len = len(synthetic_list)

      if(prev_len != up_len):
        stop = False

    if(stop == True):
      force_stop = True

    # break
  return synthetic_list_augmented , synthetic_list

def prepare_synthetic_dataset(theme_df, theme, length, join=True, mix=True):
  # length --> will denote no of EXTRA ROWS WE NEED
  if(length <= 0):
    return theme_df, theme_df

  synthetic_list_augmented, synthetic_list = Generate_Synthetic_Data(theme_df, theme, length )      #1000 - len(theme_df)
  synthetic_df = pd.DataFrame(synthetic_list)

  if(join == True):
    for ind in range(len(theme_df)):
      synthetic_df.loc[len(synthetic_df)]= theme_df.iloc[ind]

  if(mix == True):
    synthetic_df = synthetic_df.sample(frac = 1)

  synthetic_df_augmented = pd.DataFrame(synthetic_list_augmented)
  if(join == True):
    for ind in range(len(theme_df)):
      synthetic_df_augmented.loc[len(synthetic_df_augmented)]= theme_df.iloc[ind]

  if(mix == True):
    synthetic_df_augmented = synthetic_df_augmented.sample(frac = 1)

  return synthetic_df, synthetic_df_augmented

# check the implementation

In [ ]:

from datasets import Dataset


# preparing the datset
def prepare_dataset(cluster_no, cluster):
  df_bytheme = pd.DataFrame(columns = ["id", "title", "context", "question", "answers"])
  for index, rows in df_squad_v1.iterrows():
    if(rows.title in cluster):
      df_bytheme.loc[len(df_bytheme.index)] = df_squad_v1.iloc[index]
  df_bytheme_syn, df_bytheme_aug = prepare_synthetic_dataset(df_bytheme, f"cluster_{cluster_no}", 3000)
  print(df_bytheme_aug)
  dataset_bytheme = Dataset.from_pandas(df_bytheme_aug)
  print(df_bytheme_aug)
  return dataset_bytheme




In [ ]:
# hyperparameters
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
pad_on_right = tokenizer.padding_side=="right"
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context when splitting is performed.


!pip install Jinja2
from huggingface_hub import whoami, create_repo
user = whoami()['name']
from pathlib import Path

from huggingface_hub import ModelCard, ModelCardData


# running the actual model
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples




def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

from tqdm.auto import tqdm

def postprocess_qa_predictions(
    examples,
    features,
    all_start_logits,
    all_end_logits,
    n_best_size=20,
    max_answer_length=30,
):
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(
        f"Post-processing {len(examples)} example predictions split into {len(features)} features."
    )

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(
                tokenizer.cls_token_id
            )
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                -1 : -n_best_size - 1 : -1
            ].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or not offset_mapping[start_index]
                        or not offset_mapping[end_index]
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                            "Starts" : start_char,
                            "Ends" : end_char
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
                0
            ]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0, "Starts":-1, "Ends": -1}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = {"text_answer" : best_answer["text"] ,
                "starts" :  best_answer["Starts"]  ,
                "ends" : best_answer["Ends"] ,
                }
        else:
            answer = {
                "text_answer" : best_answer["text"] if best_answer["score"] > min_null_score else "",
                "starts" :  best_answer["Starts"] if best_answer["score"] > min_null_score else -1 ,
                "ends" : best_answer["Ends"] if best_answer["score"] > min_null_score else -1,

            }
            predictions[example["id"]] = answer


    return predictions

def run_model(datasets, cluster_no):
  # splitting the dataset
  datasets = datasets.train_test_split(test_size=0.2)
  tokenized_datasets = datasets.map(
    prepare_train_features, batched=True, remove_columns=datasets["train"].column_names
  )

  model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

  # trainging params
  push_to_hub_model_id = f"{cluster_no}-clustered_aug"
  learning_rate = 2e-5
  num_train_epochs = 2
  weight_decay = 0.01

  # preparing the data
  train_set = model.prepare_tf_dataset(
      tokenized_datasets["train"],
      shuffle=True,
      batch_size=batch_size,
  )

  validation_set = model.prepare_tf_dataset(
      tokenized_datasets["test"],   # changed to test as I named the split column test
      shuffle=False,
      batch_size=batch_size,
  )


  # calculating training steps
  total_train_steps = len(train_set) * num_train_epochs

  optimizer, schedule = create_optimizer(
      init_lr=learning_rate, num_warmup_steps=0, num_train_steps=total_train_steps
  )

  model.compile(optimizer=optimizer, metrics=["accuracy"])


  card_data = ModelCardData(language='en', license='mit', library_name='keras')
  card = ModelCard.from_template(
      card_data,
      model_description=str(clustered[cluster_no]),
      developers="nandysoham",
      repo="",
  )
  user = whoami()['name']
  repo_id = f'{user}/{push_to_hub_model_id}'

  push_to_hub_callback = PushToHubCallback(
      output_dir=f"./qa_model_save-{cluster_no}",
      tokenizer=tokenizer,
      hub_model_id=push_to_hub_model_id
  )

  tensorboard_callback = TensorBoard(log_dir="./qa_model_save/logs")

  callbacks = [tensorboard_callback, push_to_hub_callback]

  model.fit(
      train_set,
      validation_data=validation_set,
      epochs=num_train_epochs,
      callbacks=callbacks,
  )

  card.push_to_hub(repo_id)
  # validate model
  validation_features = datasets["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["test"].column_names,
  )

  validation_dataset = model.prepare_tf_dataset(
     validation_features,
     shuffle=False,
     batch_size=batch_size,
  )

  raw_predictions = model.predict(validation_dataset)

  examples = datasets["train"]
  features = validation_features

  example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
  features_per_example = collections.defaultdict(list)
  for i, feature in enumerate(features):
      pass
      # print(feature)
      # features_per_example[example_id_to_index[feature["example_id"]]].append(i)


  final_predictions = postprocess_qa_predictions(
    datasets["test"],
    validation_features,
    raw_predictions["start_logits"],
    raw_predictions["end_logits"],
  )


  # check accuracy
  metric = load_metric("squad_v2" if squad_v2 else "squad")

  if squad_v2:
    formatted_predictions = [
        {"id": k, "prediction_text": v["text_answer"], "no_answer_probability": 0.0}
        for k, v in final_predictions.items()
    ]
  else:
      formatted_predictions = [
          {"id": k, "prediction_text": v["text_answer"]} for k, v in final_predictions.items()
      ]
  references = [
      {"id": ex["id"], "answers": ex["answers"]} for ex in datasets["test"]
  ]
  compute = metric.compute(predictions=formatted_predictions, references=references)
  print(compute)




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def prepare_model(no):
  dataset_bycluster = prepare_dataset(no, clustered[str(no)])
  run_model(dataset_bycluster, str(no))

for i in range(19,21):
  prepare_model(i)

                            id                       title  \
3702  57264597f1498d1400e8dad1                  Film_speed   
2028        62191.378804300446                  cluster_19   
2759        20464.236057279893                  cluster_19   
2386         16845.15708836539                  cluster_19   
2118         61596.61125067668                  cluster_19   
...                        ...                         ...   
2937         12725.06903421431                  cluster_19   
3488  56fac8e58f12f319006301aa  High-definition_television   
2969         98810.81623424518                  cluster_19   
1048         78019.99129417959                  cluster_19   
2791          64622.0378080973                  cluster_19   

                                                context  \
3702  On an international level the German DIN 4512 ...   
2028  Relatively, with correspondingly lower index, ...   
2759  [? ]Postwar broadcast coverage was extended to...   
2386   More signifi

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at Rocketknight1/distilbert-base-uncased-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.
Cloning https://huggingface.co/nandysoham16/19-clustered_aug into local empty directory.


Download file tf_model.h5:   0%|          | 1.43k/253M [00:00<?, ?B/s]

...

Epoch 1/2
319/319 [==============================] - 239s 732ms/step - loss: 1.0552 - end_logits_accuracy: 0.7055 - start_logits_accuracy: 0.6754 - val_loss: 0.9062 - val_end_logits_accuracy: 0.7467 - val_start_logits_accuracy: 0.7310
Epoch 2/2
319/319 [==============================] - 245s 768ms/step - loss: 0.6589 - end_logits_accuracy: 0.8072 - start_logits_accuracy: 0.7870 - val_loss: 0.8472 - val_end_logits_accuracy: 0.7694 - val_start_logits_accuracy: 0.7506


  0%|          | 0/2 [00:00<?, ?ba/s]

80/80 [==============================] - 19s 219ms/step
Post-processing 1267 example predictions split into 1275 features.


  0%|          | 0/1267 [00:00<?, ?it/s]

{'exact_match': 71.74427782162589, 'f1': 84.25493417265466}
